In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC

target = 'moved_after_2019'
idx = 'user_id'
root = '/Users/senaemirler/garanti-bbva-data-camp/'


In [2]:
df_train = pd.read_csv(os.path.join(root, 'train_users.csv'))
df_test  = pd.read_csv(os.path.join(root, 'test_users.csv'))
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_edu   = pd.read_csv(os.path.join(root, 'education.csv'))
df_skill = pd.read_csv(os.path.join(root, 'skills.csv'))
df_exp   = pd.read_csv(os.path.join(root, 'work_experiences.csv'))

In [3]:
value_counts = df_edu['degree'].value_counts().sort_values(ascending=False)
num_unique = df_edu['degree'].nunique()
print(f"There are {num_unique} unique values in the 'school_name' column.")

unique_values_degree = df_edu['degree'].unique()
for value, count in value_counts.items():
    print(f"{value}: {count}")
    

There are 5888 unique values in the 'school_name' column.
Lisans Derecesi: 19144
Bachelor's degree: 13993
Yüksek Lisans (Master): 5246
Master's degree: 4859
Lisans: 2974
Master of Science - MS: 2344
Bachelor of Science - BS: 2051
High School Diploma: 1720
Lise: 1602
High School: 1503
Bachelor of Engineering - BE: 1476
Önlisans: 1389
Associate's degree: 1381
Bachelor's Degree: 1365
Bachelor of Science (BS): 1216
Ön Lisans: 1070
Bachelor’s Degree: 959
Bachelor of Science (BSc): 896
Doctor of Philosophy - PhD: 855
BS: 797
Mühendislik Fakültesi Mezunu: 741
Master of Science (MSc): 726
Bachelor of Engineering (B.E.): 592
Master of Business Administration - MBA: 561
Mühendislik Fakültesi: 549
Bachelor of Science (B.Sc.): 539
Master’s Degree: 527
Master of Science (M.Sc.): 509
Master's Degree: 509
İşletme Yüksek Lisans Programı (MBA): 457
Licentiate degree: 436
Bachelor of Science (B.S.): 417
Engineer's degree: 404
Master: 398
Doctor of Philosophy (Ph.D.): 387
Master of Business Administratio

In [ ]:
# Define the list of terms to check for in the school_name column
list_bachelor = ['üniversite', 'university', 'uni', 'üni', 'fakülte', 'faculty', 'enstitü', 'enstitude']


# Create a boolean mask to filter rows with the terms in my_list in the school_name column
mask = df_edu['school_name'].str.contains('|'.join(list_bachelor))

# Set the value of the degree column to 'bachelor' for the filtered rows with a blank degree
df_edu.loc[mask & df_edu['degree'].isnull(), 'degree'] = 'graduate'

In [ ]:
import re

df_edu['degree'] = df_edu['degree'].apply(lambda x: 'other' if re.search('ortaöğretim|ortaokul|highschool|ortaöğretim|ortaöğretimLİSE|summer|certificate|dil eğitimi|High School|i̇.ö.o|orta okul|i̇smek|camp|kamp|okulları|bilge adam |koleji|koleji|udacity|i̇lköğretim|bilge adam|kolej|ilk ögretim|ilkogretim|i̇lkokul|lse|ilköğretim|ilkolul|lise|high school', str(x).lower()) else x)



In [ ]:
df_edu.fillna({'degree':'other'}, inplace=True)

In [ ]:
edu_conv = {
    "(?i).*(doktora|doctor|phd|ph.d|Ph. D.|Ph. D|ph. d.|ph. d).*": "phd",
    "(?i).*(yüksek|master|M.Sc|MS|MBA|M.S).*": "master",
    "(?i).*(license|computer|önlisan|associate|becholar|muh|müh|lisans|licence|bachelor|BS|B.S|B.Sc|engineer|mühendis|mezun|graduate|licentiate|4|BA|ba|fakülte|faculty|bilgisayar|data).*": "graduate",
}

df_edu.loc[:, 'degree'] = df_edu.loc[:, 'degree'].replace(edu_conv, regex=True)
df_edu = df_edu[df_edu['degree'].isin(edu_conv.values())]

In [ ]:
value_counts = df_edu['degree'].value_counts().sort_values(ascending=False)
num_unique = df_edu['degree'].nunique()
print(f"There are {num_unique} unique values in the 'school_name' column.")

unique_values_degree = df_edu['degree'].unique()
for value, count in value_counts.items():
    print(f"{value}: {count}")
    

In [ ]:
unique_values_location = df_edu['degree'].unique()

location_list = []
for value in unique_values_location:
    if 'önlisan' in value.lower():
        location = value.strip().title()
        location_list.append(location)

print(location_list)


In [ ]:
# 'fields_of_study' sütunundaki değerlerin sayısını sayma
counts = df_edu['degree'].value_counts()

# 20'den az tekrarlanan değerleri seçme
rare_values = counts[counts < 209].index.tolist()

# 'other' olarak değiştirme
df_edu.loc[df_edu['degree'].isin(rare_values), 'degree'] = 'other'